<a href="https://colab.research.google.com/github/BheezPen/Tolaram_ProjectV2/blob/main/TOLARAM_AI_AGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install --quiet smolagents[toolkit] openai gradio
!pip install --quiet smolagents[toolkit,litellm] openai gradio

In [2]:
# Assuming these imports are at the top of your Colab notebook
import os
import getpass
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # Added for enhanced plotting
from smolagents import CodeAgent, LiteLLMModel, Tool
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import datetime # Import datetime for date operations
import io # Added for info() tool
from itertools import combinations # Added for pair analysis
from collections import Counter # Added for pair analysis
from sklearn.metrics.pairwise import cosine_similarity # Added for recommender
from sklearn.preprocessing import OneHotEncoder # Added for recommender

In [3]:
# @st.cache_data (from app.py) - Replicated for agent's context
def load_sales_data_for_agent():
    try:
        # Specify 'Redistribution Value' column as string type
        df_loaded = pd.read_csv("data_sample_analysis_cleaned.csv", encoding='latin1', dtype={'Redistribution Value': str})
    except FileNotFoundError:
        print("data_sample_analysis_cleaned.csv not found. Please ensure it's uploaded or path is correct.")
        return pd.DataFrame() # Return empty DataFrame on error

    df_loaded['Redistribution Value'] = (
        df_loaded['Redistribution Value']
        .str.replace(',', '', regex=False)
        .astype(float)
    )
    df_loaded['Delivered_date'] = pd.to_datetime(
        df_loaded['Delivered_date'], errors='coerce', dayfirst=True
    )
    df_loaded['Month'] = df_loaded['Delivered_date'].dt.to_period('M')
    df_loaded['Delivered Qty'] = df_loaded['Delivered Qty'].fillna(0)
    df_loaded['Total_Amount_Spent'] = df_loaded['Redistribution Value'] * df_loaded['Delivered Qty']
    if 'Order_Id' not in df_loaded.columns:
        # Generate a unique Order_Id if missing, similar to Streamlit app
        df_loaded['Order_Id'] = df_loaded['Customer_Phone'].astype(str) + '_' + \
                                df_loaded['Delivered_date'].dt.strftime('%Y%m%d%H%M%S') + '_' + \
                                df_loaded.groupby(['Customer_Phone', 'Delivered_date']).cumcount().astype(str)
    return df_loaded

# @st.cache_data (from app.py) - Replicated for agent's context
def load_model_preds_for_agent():
    try:
        preds_loaded = pd.read_csv(
            "purchase_predictions_major.csv",
            parse_dates=["last_purchase_date", "pred_next_date"],
        )
    except FileNotFoundError:
        print("purchase_predictions_major.csv not found. Please ensure it's uploaded or path is correct.")
        return pd.DataFrame() # Return empty DataFrame on error

    preds_loaded = preds_loaded.rename(columns={
        "pred_next_brand":     "Next Brand Purchase",
        "pred_next_date":      "Next Purchase Date",
        "pred_spend":          "Expected Spend",
        "pred_qty":            "Expected Quantity",
        "probability":         "Probability"
    })
    preds_loaded["Next Purchase Date"] = preds_loaded["Next Purchase Date"].dt.date
    preds_loaded["Expected Spend"] = preds_loaded["Expected Spend"].round(0).astype(int)
    preds_loaded["Expected Quantity"] = preds_loaded["Expected Quantity"].round(0).astype(int)
    preds_loaded["Probability"] = (preds_loaded["Probability"] * 100).round(1)
    def suggest(p):
        if p >= 70:
            return "Follow-up/Alert"
        if p >= 50:
            return "Cross Sell"
        return "Discount"
    preds_loaded["Suggestion"] = preds_loaded["Probability"].apply(suggest)
    return preds_loaded

# Load data globally for the agent's environment
df = load_sales_data_for_agent()
PRED_DF = load_model_preds_for_agent()

print("DataFrame 'df' is ready. Here are the first few rows:")
print(df.head())
if not PRED_DF.empty:
    print("\nDataFrame 'PRED_DF' is ready. Here are the first few rows:")
    print(PRED_DF.head())
else:
    print("\nPRED_DF is empty or not loaded. Some prediction functionalities may be limited.")

# --- Helper functions (from app.py, used by multiple tools) ---
def calculate_brand_sku_pairs_internal(data_frame, type_col='Brand'):
    """
    Calculates the frequency of item pairs (Brand or SKU) appearing together in orders.
    Args:
        data_frame (pd.DataFrame): The input DataFrame with sales data.
        type_col (str): 'Brand' or 'SKU_Code' to specify what to pair.
    Returns:
        pd.DataFrame: A DataFrame containing the top pairs and their co-occurrence counts.
    """
    if 'Order_Id' not in data_frame.columns:
        # This should ideally be handled during initial data loading, but as a fallback
        data_frame['Order_Id'] = data_frame['Customer_Phone'].astype(str) + "_" + \
                                 data_frame['Delivered_date'].dt.strftime('%Y%m%d%H%M%S') + '_' + \
                                 data_frame.groupby(['Customer_Phone', 'Delivered_date']).cumcount().astype(str)

    order_items = data_frame.groupby("Order_Id")[type_col].apply(set)
    pair_counts = Counter()
    for items in order_items:
        if len(items) > 1:
            for pair in combinations(items, 2):
                pair_counts[tuple(sorted(pair))] += 1

    pair_df = pd.DataFrame(pair_counts.items(), columns=[f"{type_col}_Pair_Tuple", "Count"]).sort_values(by="Count", ascending=False)

    if type_col == 'SKU_Code':
        sku_to_brand = data_frame.groupby('SKU_Code')['Brand'].first().to_dict()
        pair_df[f"{type_col}_Pair_Formatted"] = pair_df[f"{type_col}_Pair_Tuple"].apply(
            lambda x: f"{x[0]} ({sku_to_brand.get(x[0], 'Unknown')}) & {x[1]} ({sku_to_brand.get(x[1], 'Unknown')})"
        )
    else: # Brand
        pair_df[f"{type_col}_Pair_Formatted"] = pair_df[f"{type_col}_Pair_Tuple"].apply(lambda x: f"{x[0]} & {x[1]}")

    return pair_df

DataFrame 'df' is ready. Here are the first few rows:
    Brand   Branch      Designation  SKU_Code Salesman_Code  \
0  ADDMIE  LAGOS 1  FIELD EXECUTIVE  10002778        807259   
1  ADDMIE  LAGOS 1  FIELD EXECUTIVE  10002778        807881   
2  ADDMIE  LAGOS 1  FIELD EXECUTIVE  10002778        807881   
3  ADDMIE  LAGOS 1  FIELD EXECUTIVE  10002778        807881   
4  ADDMIE  LAGOS 1  FIELD EXECUTIVE  10002778        807881   

                   Salesman_Name    Customer_Name  Customer_Phone  Order_Id  \
0  ANOSIKE MIRACLE CHIMECHEFULAM        Eze store      8060733751  11432770   
1                  OKERE IFEANYI          Chimaka      9016204667  11565436   
2                  OKERE IFEANYI          Chimaka      9016204667  11727113   
3                  OKERE IFEANYI      Cynthia chi      8060359775  11756045   
4                  OKERE IFEANYI  Daddy Josephine      9075013565  11585122   

  Delivered_date  Delivered Qty  Redistribution Value    Month  \
0     2024-05-08          

<ipython-input-3-3f79dda6c883>:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_loaded = pd.read_csv("data_sample_analysis_cleaned.csv", encoding='latin1', dtype={'Redistribution Value': str})


In [4]:
# --- Tool Definitions (Existing and New) ---

# 1) HeadTool: Return the first n rows of the DataFrame df.
class HeadTool(Tool):
    name = "head"
    description = "Return the first n rows of the DataFrame df."
    inputs = {"n": {"type": "integer", "description": "Number of rows to display"}}
    output_type = "object" # pandas.DataFrame
    def forward(self, n: int):
        return df.head(n)

# 2) TailTool: Return the last n rows of the DataFrame 'df'.
class TailTool(Tool):
    name = "tail"
    description = "Return the last n rows of the DataFrame 'df'."
    inputs = {"n": {"type": "integer", "description": "Number of rows to display"}}
    output_type = "object"
    def forward(self, n: int):
        return df.tail(n)

# 3) DescribeTool: Summary statistics for a column or entire df
class DescribeTool(Tool):
    name = "describe"
    description = (
        "Return summary statistics. If a column is given, describe that column; "
        "otherwise, describe the entire DataFrame."
    )
    inputs = {
        "column": {
            "type": "string",
            "description": "Name of column to describe ('all' for full df)",
            "nullable": True,
            "required": False,
        }
    }
    output_type = "object" # pandas.DataFrame or pandas.Series
    def forward(self, column: str = None):
        if column is None or column.lower() in ("all", ""):
            return df.describe(include="all")
        else:
            if column not in df.columns:
                raise ValueError(f"Column '{column}' does not exist in df")
            return df[column].describe()

# 4) InfoTool: Return df.info() string
class InfoTool(Tool):
    name = "info"
    description = "Return the output of df.info() as a string."
    inputs = {}
    output_type = "string"
    def forward(self):
        import io
        buf = io.StringIO()
        df.info(buf=buf)
        return buf.getvalue() # Return the string value

# 5) HistogramTool: Plot a histogram of a numeric column
class HistogramTool(Tool):
    name = "histogram"
    description = (
        "Plot a histogram of a numeric column in df. Returns 'PLOTTED' "
        "after showing the plot."
    )
    inputs = {
        "column": {"type": "string", "description": "Name of the numeric column to plot"},
        "bins": {
            "type": "integer",
            "description": "Number of bins for the histogram (optional)",
            "required": False,
            "nullable": True,
        },
    }
    output_type = "string"
    def forward(self, column: str, bins: int = 10):
        if column not in df.columns:
            raise ValueError(f"Column '{column}' does not exist in df")
        try:
            series = df[column].dropna().astype(float)
        except Exception:
            raise ValueError(f"Column '{column}' cannot be converted to numeric for histogram")

        plt.figure(figsize=(8, 6))
        sns.histplot(series, bins=bins, kde=True, color="forestgreen")
        plt.title(f"Histogram of '{column}'")
        plt.xlabel(column)
        plt.ylabel("Frequency")
        plt.grid(axis='y', alpha=0.5)
        plt.tight_layout()
        plt.show()
        return "PLOTTED"

# 6) ScatterPlotTool: Plot a scatter of two numeric columns
class ScatterPlotTool(Tool):
    name = "scatter_plot"
    description = (
        "Plot a scatter plot of two numeric columns. Returns 'PLOTTED' after showing the plot."
    )
    inputs = {
        "column_x": {"type": "string", "description": "Name of the numeric column for the x-axis"},
        "column_y": {"type": "string", "description": "Name of the numeric column for the y-axis"},
    }
    output_type = "string"
    def forward(self, column_x: str, column_y: str):
        if column_x not in df.columns or column_y not in df.columns:
            raise ValueError(f"Columns '{column_x}' or '{column_y}' not found in df")
        try:
            x = df[column_x].dropna().astype(float)
            y = df[column_y].dropna().astype(float)
        except Exception:
            raise ValueError("Columns must be numeric for scatter plot")

        plt.figure(figsize=(8, 6))
        plt.scatter(x, y, alpha=0.6)
        plt.title(f"Scatter: {column_x} vs {column_y}")
        plt.xlabel(column_x)
        plt.ylabel(column_y)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        return "PLOTTED"

# 7) CorrelationTool: Compute correlation matrix of numeric columns
class CorrelationTool(Tool):
    name = "correlation"
    description = (
        "Compute the pairwise correlation matrix of numeric columns in df. "
        "Returns a pandas.DataFrame of correlations."
    )
    inputs = {
        "method": {
            "type": "string",
            "description": "Correlation method: 'pearson' or 'spearman' (optional)",
            "required": False,
            "nullable": True,
        }
    }
    output_type = "object" # pandas.DataFrame
    def forward(self, method: str = "pearson"):
        if method not in ("pearson", "spearman"):
            raise ValueError("Method must be 'pearson' or 'spearman'")
        numeric_df = df.select_dtypes(include=[np.number])
        return numeric_df.corr(method=method)

# 8) PivotTableTool: Create a pivot table from df
class PivotTableTool(Tool):
    name = "pivot_table"
    description = (
        "Create a pivot table. Specify index, columns, values, and aggregation function. "
        "Returns a pandas.DataFrame."
    )
    inputs = {
        "index": {"type": "string", "description": "Column name to use as the pivot index"},
        "columns": {"type": "string", "description": "Column name to use as the pivot columns"},
        "values": {
            "type": "string",
            "description": "Column name(s) to use as values (if multiple, separate by commas)",
        },
        "aggfunc": {
            "type": "string",
            "description": "Aggregation function: 'sum', 'mean', 'count', 'max', or 'min'",
        },
    }
    output_type = "object" # pandas.DataFrame
    def forward(self, index: str, columns: str, values: str, aggfunc: str):
        if index not in df.columns or columns not in df.columns:
            raise ValueError(f"Index '{index}' or columns '{columns}' not found in df")
        vals = [v.strip() for v in values.split(",")]
        for v in vals:
            if v not in df.columns:
                raise ValueError(f"Value column '{v}' not found in df")
        if aggfunc not in ("sum", "mean", "count", "max", "min"):
            raise ValueError("aggfunc must be one of 'sum', 'mean', 'count', 'max', 'min'")
        pivot = pd.pivot_table(df, index=index, columns=columns, values=vals, aggfunc=aggfunc)
        return pivot

# 9) FilterRowsTool: Filter df based on a comparison
class FilterRowsTool(Tool):
    name = "filter_rows"
    description = (
        "Filter rows from df based on a comparison column, operator, value. "
        "Returns the filtered DataFrame."
    )
    inputs = {
        "column": {"type": "string", "description": "Column name to apply filter on"},
        "operator": {
            "type": "string",
            "description": "Comparison operator: one of '>', '<', '=', '>=', '<=', '=='",
        },
        "value": {
            "type": "string",
            "description": (
                "Value to compare to (if numeric, will be parsed). "
                "If string column, wrap value in quotes."
            ),
        },
    }
    output_type = "object"
    def forward(self, column: str, operator: str, value: str):
        if column not in df.columns:
            raise ValueError(f"Column '{column}' not found in df")

        # Attempt numeric cast; if fails, treat as string
        try:
            test_val = float(value)
            ser = df[column].astype(float)
        except ValueError:
            test_val = value.strip().strip("'").strip('"') # Remove quotes for string comparison
            ser = df[column].astype(str)

        if operator == ">":
            mask = ser > test_val
        elif operator == "<":
            mask = ser < test_val
        elif operator == ">=":
            mask = ser >= test_val
        elif operator == "<=":
            mask = ser <= test_val
        elif operator == "==":
            mask = ser == test_val
        elif operator == "!=":
            mask = ser != test_val
        else:
            raise ValueError(f"Operator '{operator}' not supported")
        return df[mask] # Returns a filtered copy of the global df

# 10) GroupByAggregateTool: Group by column(s) and apply aggregation
class GroupByAggregateTool(Tool):
    name = "groupby_agg"
    description = (
        "Group the DataFrame by one or more columns, then aggregate a metric column "
        "using a specified function. Returns the aggregated DataFrame."
    )
    inputs = {
        "group_columns": {"type": "string", "description": "Comma-separated column names to group by"},
        "metric_column": {"type": "string", "description": "Name of the numeric column to aggregate"},
        "aggfunc": {
            "type": "string",
            "description": "Aggregation function: 'sum', 'mean', 'count', 'max', or 'min'",
        },
    }
    output_type = "object" # pandas.DataFrame or Series
    def forward(self, group_columns: str, metric_column: str, aggfunc: str):
        groups = [c.strip() for c in group_columns.split(",")]
        for c in groups:
            if c not in df.columns:
                raise ValueError(f"Group-by column '{c}' not found in df")
        if metric_column not in df.columns:
            raise ValueError(f"Metric column '{metric_column}' not found in df")
        if aggfunc not in ("sum", "mean", "count", "max", "min"):
            raise ValueError("aggfunc must be one of 'sum', 'mean', 'count', 'max', 'min'")
        grouped = df.groupby(groups)[metric_column].agg(aggfunc).reset_index()
        return grouped

# 11) SortTool: Sort df by a specified column, ascending or descending
class SortTool(Tool):
    name = "sort"
    description = (
        "Sort the DataFrame by a specified column. "
        "Specify ascending (True/False). Returns the sorted DataFrame."
    )
    inputs = {
        "column": {"type": "string", "description": "Column name to sort by"},
        "ascending": {
            "type": "boolean",
            "description": "Sort order: True for ascending, False for descending",
        },
    }
    output_type = "object"
    def forward(self, column: str, ascending: bool):
        if column not in df.columns:
            raise ValueError(f"Column '{column}' not found in df")
        return df.sort_values(by=column, ascending=ascending)

# 12) TopNTool: Return top N rows after grouping/aggregation or simple sort (Updated)
class TopNTool(Tool):
    name = "top_n"
    description = (
        "Return the top N rows by a given metric. If group_columns is provided, "
        "it groups by those columns, aggregates metric_column by sum, then returns "
        "the top N groups. Otherwise, it simply sorts df by metric_column and returns top N rows."
        "Specify ascending (True/False) for sorting order (True for bottom N, False for top N)."
    )
    inputs = {
        "metric_column": {"type": "string", "description": "Name of the numeric column to rank by"},
        "n": {"type": "integer", "description": "Number of top/bottom rows/groups to return"},
        "group_columns": {
            "type": "string",
            "description": "Comma-separated columns to group by (optional)",
            "required": False,
            "nullable": True,
        },
        "ascending": {
            "type": "boolean",
            "description": "Sort order: True for ascending (bottom N), False for descending (top N). Default is False.",
            "required": False,
            "nullable": True,
        }
    }
    output_type = "object" # pandas.DataFrame
    def forward(self, metric_column: str, n: int, group_columns: str = None, ascending: bool = False):
        if metric_column not in df.columns:
            raise ValueError(f"Metric column '{metric_column}' not found in df")

        if group_columns is None or not group_columns.strip():
            return df.sort_values(by=metric_column, ascending=ascending).head(n)
        else:
            groups = [c.strip() for c in group_columns.split(",")]
            for c in groups:
                if c not in df.columns:
                    raise ValueError(f"Group-by column '{c}' not found in df")

            grouped = df.groupby(groups)[metric_column].sum().reset_index()
            return grouped.sort_values(by=metric_column, ascending=ascending).head(n)

# 13) CrosstabTool: Crosstab between two categorical columns
class CrosstabTool(Tool):
    name = "crosstab"
    description = (
        "Compute a cross-tabulation (frequency table) between two categorical columns. "
        "Returns a pandas.DataFrame."
    )
    inputs = {
        "row": {"type": "string", "description": "Column name for rows"},
        "column": {"type": "string", "description": "Column name for columns"},
        "aggfunc": {
            "type": "string",
            "description": "Aggregation function: 'count', 'sum', 'mean' (optional)",
            "required": False,
            "nullable": True,
        },
        "values": {
            "type": "string",
            "description": "Name of value column if aggfunc is not None (optional)",
            "required": False,
            "nullable": True,
        },
    }
    output_type = "object"
    def forward(self, row: str, column: str, aggfunc: str = None, values: str = None):
        if row not in df.columns or column not in df.columns:
            raise ValueError(f"Columns '{row}' or '{column}' not found in df")

        if aggfunc:
            if values is None:
                raise ValueError("Please supply 'values' when using an aggregation function")
            if values not in df.columns:
                raise ValueError(f"Values column '{values}' not found in df")
            if aggfunc not in ("sum", "mean", "count"):
                raise ValueError("aggfunc must be one of 'sum', 'mean', 'count'")
            return pd.crosstab(df[row], df[column], values=df[values], aggfunc=aggfunc)
        else:
            return pd.crosstab(df[row], df[column])

# 14) LinRegEvalTool: Train/test split linear regression + R² metrics
class LinRegEvalTool(Tool):
    name = "linreg_eval"
    description = (
        "Split df into train/test, train a LinearRegression model, and return R² on both sets. "
        "feature_columns: comma-separated list of features; target_column: name of target; "
        "test_size: fraction for test (optional, default 0.2). "
        "Returns a pandas.DataFrame with 'train' and 'test' R²."
    )
    inputs = {
        "feature_columns": {"type": "string", "description": "Comma-separated column names to use as features"},
        "target_column": {"type": "string", "description": "Name of the target column"},
        "test_size": {
            "type": "number",
            "description": "Fraction of data to use as test (optional; default 0.2)",
            "required": False,
            "nullable": True,
        },
    }
    output_type = "object"
    def forward(self, feature_columns: str, target_column: str, test_size: float = 0.2):
        feats = [c.strip() for c in feature_columns.split(",")]
        for c in feats:
            if c not in df.columns:
                raise ValueError(f"Feature column '{c}' not in df")
        if target_column not in df.columns:
            raise ValueError(f"Target column '{target_column}' not in df")

        sub = df[feats + [target_column]].dropna()
        X = sub[feats].values
        y = sub[target_column].values.astype(float)

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42
        )

        model = LinearRegression()
        model.fit(X_train, y_train)

        r2_train = r2_score(y_train, model.predict(X_train))
        r2_test = r2_score(y_test, model.predict(X_test))

        metrics_df = pd.DataFrame({"r2": [r2_train, r2_test]}, index=["train", "test"])
        return metrics_df

# 15) PredictLinearTool: Train on full df + predict for a new sample
class PredictLinearTool(Tool):
    name = "predict_linear"
    description = (
        "Train a LinearRegression model on the entire df using feature_columns "
        "then predict on a new row. feature_columns: comma-separated features; "
        "target_column: name of target; new_data: comma-separated numeric values. "
        "Returns the predicted numeric value."
    )
    inputs = {
        "feature_columns": {"type": "string", "description": "Comma-separated column names to use as features"},
        "target_column": {"type": "string", "description": "Name of the target column"},
        "new_data": {
            "type": "string",
            "description": (
                "Comma-separated numeric values for the new sample, in same order as feature_columns"
            ),
        },
    }
    output_type = "number"
    def forward(self, feature_columns: str, target_column: str, new_data: str):
        feats = [c.strip() for c in feature_columns.split(",")]
        if target_column not in df.columns:
            raise ValueError(f"Target column '{target_column}' not in df")
        for c in feats:
            if c not in df.columns:
                raise ValueError(f"Feature column '{c}' not in df")

        values = [float(x.strip()) for x in new_data.split(",")]
        if len(values) != len(feats):
            raise ValueError("Number of new_data values must match number of features")

        sub = df[feats + [target_column]].dropna()
        X = sub[feats].values
        y = sub[target_column].values.astype(float)

        model = LinearRegression()
        model.fit(X, y)

        pred = model.predict(np.array(values).reshape(1, -1))[0]
        return pred

# 16) RFClassifyTool: Train/test split Random Forest classification
class RFClassifyTool(Tool):
    name = "rf_classify"
    description = (
        "Split df into train/test, train a RandomForestClassifier, and return classification report. "
        "feature_columns: comma-separated features; target_column: name of target class; "
        "test_size: fraction for test (optional, default 0.2). "
        "Returns a classification report dictionary."
    )
    inputs = {
        "feature_columns": {"type": "string", "description": "Comma-separated column names to use as features"},
        "target_column": {"type": "string", "description": "Name of the target class column"},
        "test_size": {
            "type": "number",
            "description": "Fraction of data to use as test (optional; default 0.2)",
            "required": False,
            "nullable": True,
        },
        "n_estimators": {
            "type": "integer",
            "description": "Number of trees in the forest (optional; default 100)",
            "required": False,
            "nullable": True,
        },
    }
    output_type = "object" # returns a dict (classification report)
    def forward(
        self, feature_columns: str, target_column: str, test_size: float = 0.2, n_estimators: int = 100
    ):
        feats = [c.strip() for c in feature_columns.split(",")]
        for c in feats:
            if c not in df.columns:
                raise ValueError(f"Feature column '{c}' not in df")
        if target_column not in df.columns:
            raise ValueError(f"Target column '{target_column}' not in df")

        sub = df[feats + [target_column]].dropna()
        X = sub[feats].values
        y = sub[target_column].values

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42
        )

        clf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
        clf.fit(X_train, y_train)

        preds = clf.predict(X_test)
        report = classification_report(y_test, preds, output_dict=True)
        acc = accuracy_score(y_test, preds)
        report["accuracy"] = acc
        return report

class FinalAnswerTool(Tool):
    name = "final_answer"
    description = "Use this to return a direct final answer to the user as a string."
    inputs = {"text": {"type": "string", "description": "Final answer to return to the user."}}
    output_type = "string"
    def forward(self, text: str):
        return text


# 18) InsightsTool: Compute overall sales dataset insights and generate actionable recommendations. (Updated)
class InsightsTool(Tool):
    name = "insights"
    description = (
        "Compute overall sales dataset insights and generate actionable recommendations."
        "Returns a string summary covering:\n"
        "Top 5 brands by revenue, \n"
        "Any brands with significant MoM drops, \n"
        "Top 5 customers by lifetime value, \n"
        "High-confidence next-purchase predictions, \n"
        "Co-purchase patterns, \n"
        "Actionable recommendations.\n"
        "No inputs required."
    )
    # Change inputs from tuple () to empty dictionary {}
    inputs = {}
    output_type = "string"

    def forward(self):
        # Ensure 'df' is accessible here. If 'PRED_DF' is also available, use it where appropriate.
        # Assuming 'df' is the main DataFrame.

        # Calculate Top 5 Brands by Revenue
        try:
            df['Redistribution Value'] = pd.to_numeric(df['Redistribution Value'], errors='coerce').fillna(0)
            top5_brands = df.groupby("Brand")["Redistribution Value"].sum().nlargest(5)
            top5_summary = "\n".join(
                [f"{(i+1)}. {brand} (Rev: {rev:,.0f})" for i, (brand, rev) in enumerate(top5_brands.items())]
            )
        except Exception as e:
            top5_summary = f"Could not calculate top 5 brands: {e}"

        # Calculate Month-over-Month Drops
        drop_insight = "Month-over-month drop analysis not performed due to data or processing issues."
        try:
            temp_df_for_dates = df.copy()
            temp_df_for_dates['Delivered_date'] = pd.to_datetime(temp_df_for_dates['Delivered_date'], errors='coerce')
            temp_df_for_dates['Month'] = temp_df_for_dates['Delivered_date'].dt.to_period('M')
            temp_df_for_dates.dropna(subset=['Month', 'Redistribution Value'], inplace=True)

            brand_month_rev = temp_df_for_dates.groupby(["Brand", "Month"])["Redistribution Value"].sum().unstack(fill_value=0)
            mom_pct = brand_month_rev.pct_change(axis=1) * 100
            drops = mom_pct.stack().reset_index(name="MoM%").query("`MoM%` < -10")

            if not drops.empty:
                drops_list = drops.groupby("Brand")["MoM%"].min().nsmallest(3)
                drops_summary = "\n".join(
                    [f"- {brand}: {pct:.1f}% drop" for brand, pct in drops_list.items()]
                )
                drop_insight = f"Brands with >10% month-over-month revenue drop (top 3 worst):\n{drops_summary}"
            else:
                drop_insight = "No brands have experienced a significant month-over-month revenue drop (greater than 10%)."
        except Exception as e:
            drop_insight = f"Could not perform MoM analysis: {e}"

        # Calculate Top 5 Customers by Lifetime Value
        try:
            df['Redistribution Value'] = pd.to_numeric(df['Redistribution Value'], errors='coerce').fillna(0)
            cust_ltv = df.groupby("Customer_Phone")["Redistribution Value"].sum().nlargest(5)
            cust_names = df.drop_duplicates("Customer_Phone").set_index("Customer_Phone")["Customer_Name"]
            top5_cust_summary = "\n".join(
                [f"{(i+1)}. {cust_names.get(phone, 'Unknown')} ({phone}) (Spend: {spend:,.0f})"
                 for i, (phone, spend) in enumerate(cust_ltv.items())]
            )
        except Exception as e:
            top5_cust_summary = f"Could not calculate top 5 customers: {e}"

        # Next-Purchase Predictions (from PRED_DF)
        pred_insight = "Next-purchase predictions not available or PRED_DF not correctly structured/defined."
        if 'PRED_DF' in globals() and not PRED_DF.empty and 'Probability' in PRED_DF.columns:
            try:
                top_pred = PRED_DF.sort_values("Probability", ascending=False).head(3)[
                    ["Customer_Phone", "Next Brand Purchase", "Probability"]
                ]
                pred_list = "\n".join(
                    [f"- {row['Customer_Phone']} likely to buy {row['Next Brand Purchase']} ({row['Probability']:.1f}%)"
                     for _, row in top_pred.iterrows()]
                )
                pred_insight = f"Top 3 next-purchase high-confidence predictions:\n{pred_list}"
            except Exception as e:
                pred_insight = f"Could not generate next-purchase predictions: {e}"
        else:
            pred_insight = "PRED_DF not found or not structured for next-purchase predictions."

        # Co-purchase Patterns (using the internal helper function)
        pair_insight = "Co-purchase patterns not found or calculation failed."
        try:
            pairs = calculate_brand_sku_pairs_internal(df, type_col='Brand')
            top_pair = pairs.head(1)
            if not top_pair.empty:
                pair_data = top_pair.iloc[0]
                pair_insight = f"Most frequently co-purchased brands: {pair_data['Brand_Pair_Formatted']} (Count: {pair_data['Count']})"
            else:
                pair_insight = "No co-purchase patterns found."
        except Exception as e:
            pair_insight = f"Could not calculate co-purchase patterns: {e}"

        # Actionable Recommendations
        recommendations = [
            "1. Consider promoting top brands with bundle discounts.",
            "2. Re-engage top customers with loyalty rewards.",
            "3. Use predicted next-purchase to trigger timely cross-sell emails.",
            "4. For very frequent brand pairs, create combo promotions.",
        ]
        if "month-over-month revenue drop" in drop_insight:
             recommendations.insert(1, "5. Investigate why certain brands are dropping (e.g., stock, pricing, competition).")


        summary = [
            "## SALES DATA INSIGHTS",
            "\n**1. Top 5 Brands by Revenue:**",
            top5_summary,
            "\n**2. Month-over-Month Drops:**",
            drop_insight,
            "\n**3. Top 5 Customers by Lifetime Value:**",
            top5_cust_summary,
            "\n**4. Next-Purchase High-Confidence Predictions (from Model):**",
            pred_insight,
            "\n**5. Most Frequent Co-Purchase Patterns (Brands):**",
            pair_insight,
            "\n**6. Actionable Recommendations:**",
            "\n".join(recommendations),
        ]

        return "\n\n".join(summary)

In [5]:
# --- NEW PLOTTING TOOLS ---

class PlotBarChartTool(Tool):
    name = "plot_bar_chart"
    description = (
        "Plots a bar chart from a DataFrame. "
        "Requires a DataFrame to plot, a column for x-axis (values), and a column for y-axis (categories/labels). "
        "Returns 'PLOTTED'."
    )
    inputs = {
        "data": {"type": "object", "description": "The DataFrame containing the data to plot."},
        "x_column": {"type": "string", "description": "The column for the x-axis (numeric values)."},
        "y_column": {"type": "string", "description": "The column for the y-axis (categorical labels)."},
        "title": {"type": "string", "description": "Title of the chart."},
        "xlabel": {"type": "string", "description": "Label for the x-axis (optional).", "required": False, "nullable": True},
        "ylabel": {"type": "string", "description": "Label for the y-axis (optional).", "required": False, "nullable": True},
        "horizontal": {"type": "boolean", "description": "Set to True for horizontal bars (default False).", "required": False, "nullable": True},
        "sort_by_x_desc": {"type": "boolean", "description": "Sort bars by x-axis value in descending order (default True).", "required": False, "nullable": True},
    }
    output_type = "string"

    def forward(self, data: pd.DataFrame, x_column: str, y_column: str, title: str, xlabel: str = None, ylabel: str = None, horizontal: bool = False, sort_by_x_desc: bool = True):
        if data.empty:
            print("Provided DataFrame is empty, cannot plot.")
            return "PLOT_FAILED: Empty DataFrame"
        if x_column not in data.columns or y_column not in data.columns:
            raise ValueError(f"Columns '{x_column}' or '{y_column}' not found in the provided DataFrame.")

        plt.figure(figsize=(10, 7))

        plot_data = data.copy()
        if sort_by_x_desc:
            plot_data = plot_data.sort_values(by=x_column, ascending=False)

        if horizontal:
            sns.barplot(x=x_column, y=y_column, data=plot_data, palette="viridis")
        else:
            sns.barplot(x=y_column, y=x_column, data=plot_data, palette="viridis")
            plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for vertical bars

        plt.title(title, fontsize=16)
        plt.xlabel(xlabel if xlabel else x_column, fontsize=13)
        plt.ylabel(ylabel if ylabel else y_column, fontsize=13)
        plt.tight_layout()
        plt.show()
        return "PLOTTED"

class PlotLineChartTool(Tool):
    name = "plot_line_chart"
    description = (
        "Plots a line chart from a DataFrame, typically for time-series data. "
        "Requires a DataFrame, a column for x-axis (time), and a column for y-axis (value). "
        "Optional: hue column for multiple lines. Returns 'PLOTTED'."
    )
    inputs = {
        "data": {"type": "object", "description": "The DataFrame containing the data to plot."},
        "x_column": {"type": "string", "description": "The column for the x-axis (e.g., 'Month', 'Date')."},
        "y_column": {"type": "string", "description": "The column for the y-axis (numeric value)."},
        "hue_column": {"type": "string", "description": "Optional: Column to create multiple lines (e.g., 'Brand', 'Customer').", "required": False, "nullable": True},
        "title": {"type": "string", "description": "Title of the chart."},
        "xlabel": {"type": "string", "description": "Label for the x-axis (optional).", "required": False, "nullable": True},
        "ylabel": {"type": "string", "description": "Label for the y-axis (optional).", "required": False, "nullable": True},
    }
    output_type = "string"

    def forward(self, data: pd.DataFrame, x_column: str, y_column: str, title: str, hue_column: str = None, xlabel: str = None, ylabel: str = None):
        if data.empty:
            print("Provided DataFrame is empty, cannot plot.")
            return "PLOT_FAILED: Empty DataFrame"
        if x_column not in data.columns or y_column not in data.columns:
            raise ValueError(f"Columns '{x_column}' or '{y_column}' not found in the provided DataFrame.")
        if hue_column and hue_column not in data.columns:
            raise ValueError(f"Hue column '{hue_column}' not found in the provided DataFrame.")

        plt.figure(figsize=(12, 7))

        # Ensure x_column is treated as datetime for proper plotting if it's a period or string
        plot_data = data.copy()
        if pd.api.types.is_period_dtype(plot_data[x_column]):
            plot_data[x_column] = plot_data[x_column].dt.to_timestamp()
        elif not pd.api.types.is_datetime64_any_dtype(plot_data[x_column]):
            plot_data[x_column] = pd.to_datetime(plot_data[x_column], errors='coerce')
        plot_data.dropna(subset=[x_column], inplace=True) # Drop rows where date conversion failed

        sns.lineplot(x=x_column, y=y_column, hue=hue_column, data=plot_data, marker="o")
        plt.title(title, fontsize=16)
        plt.xlabel(xlabel if xlabel else x_column, fontsize=13)
        plt.ylabel(ylabel if ylabel else y_column, fontsize=13)
        plt.xticks(rotation=45, ha='right')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        return "PLOTTED"

class PlotDualAxisLineChartTool(Tool):
    name = "plot_dual_axis_line_chart"
    description = (
        "Plots two line charts on a dual y-axis for comparison, typically for time-series data. "
        "Requires a DataFrame, a common x-axis (time), and two different y-axes (values). "
        "Returns 'PLOTTED'."
    )
    inputs = {
        "data": {"type": "object", "description": "The DataFrame containing the data to plot."},
        "x_column": {"type": "string", "description": "The common column for the x-axis (e.g., 'Month', 'Date')."},
        "y1_column": {"type": "string", "description": "The column for the first y-axis (numeric value)."},
        "y2_column": {"type": "string", "description": "The column for the second y-axis (numeric value)."},
        "title": {"type": "string", "description": "Title of the chart."},
        "xlabel": {"type": "string", "description": "Label for the x-axis (optional).", "required": False, "nullable": True},
        "y1_label": {"type": "string", "description": "Label for the first y-axis (optional).", "required": False, "nullable": True},
        "y2_label": {"type": "string", "description": "Label for the second y-axis (optional).", "required": False, "nullable": True},
    }
    output_type = "string"

    def forward(self, data: pd.DataFrame, x_column: str, y1_column: str, y2_column: str, title: str, xlabel: str = None, y1_label: str = None, y2_label: str = None):
        if data.empty:
            print("Provided DataFrame is empty, cannot plot.")
            return "PLOT_FAILED: Empty DataFrame"
        if not all(col in data.columns for col in [x_column, y1_column, y2_column]):
            raise ValueError("One or more specified columns not found in the provided DataFrame.")

        plt.figure(figsize=(12, 7))
        fig, ax1 = plt.subplots(figsize=(14, 7))

        plot_data = data.copy()
        if pd.api.types.is_period_dtype(plot_data[x_column]):
            plot_data[x_column] = plot_data[x_column].dt.to_timestamp()
        elif not pd.api.types.is_datetime64_any_dtype(plot_data[x_column]):
            plot_data[x_column] = pd.to_datetime(plot_data[x_column], errors='coerce')
        plot_data.dropna(subset=[x_column], inplace=True)

        color_y1 = "royalblue"
        color_y2 = "darkorange"

        sns.lineplot(data=plot_data, x=x_column, y=y1_column, marker="o", label=y1_label if y1_label else y1_column, ax=ax1, color=color_y1, linewidth=2)
        ax2 = ax1.twinx()
        sns.lineplot(data=plot_data, x=x_column, y=y2_column, marker="s", label=y2_label if y2_label else y2_column, ax=ax2, color=color_y2, linewidth=2)

        ax1.set_title(title, fontsize=16)
        ax1.set_xlabel(xlabel if xlabel else x_column, fontsize=13)
        ax1.set_ylabel(y1_label if y1_label else y1_column, color=color_y1, fontsize=13)
        ax2.set_ylabel(y2_label if y2_label else y2_column, color=color_y2, fontsize=13)

        ax1.tick_params(axis='y', labelcolor=color_y1)
        ax2.tick_params(axis='y', labelcolor=color_y2)
        ax1.tick_params(axis='x', rotation=45, labelsize=12)

        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=12)
        ax2.get_legend().remove() # Remove redundant legend on the second axis

        plt.tight_layout()
        plt.show()
        return "PLOTTED"

In [6]:
# --- NEW ANALYSIS/REPORTING TOOLS ---

class BrandSKUPairAnalysisTool(Tool):
    name = "brand_sku_pair_analysis"
    description = (
        "Analyzes and plots the most frequently co-purchased brand or SKU pairs. "
        "Specify 'type' as 'Brand' or 'SKU_Code'. Returns 'PLOTTED'."
    )
    inputs = {
        "type": {"type": "string", "description": "Specify 'Brand' for brand pairs or 'SKU_Code' for SKU pairs."},
        "top_n": {"type": "integer", "description": "Number of top pairs to display (default 10).", "required": False, "nullable": True},
    }
    output_type = "string"

    def forward(self, type: str, top_n: int = 10):
        if type not in ['Brand', 'SKU_Code']:
            raise ValueError("Type must be 'Brand' or 'SKU_Code'.")

        if type == 'Brand' and 'Brand' not in df.columns:
            raise ValueError("Brand column not found in DataFrame.")
        if type == 'SKU_Code' and 'SKU_Code' not in df.columns:
            raise ValueError("SKU_Code column not found in DataFrame.")

        pair_df = calculate_brand_sku_pairs_internal(df, type_col=type)
        pair_df = pair_df.head(top_n)

        if pair_df.empty:
            print(f"No {type} pairs found.")
            return "PLOT_FAILED: No pairs found"

        plt.figure(figsize=(12, max(7, top_n * 0.7))) # Adjust height based on top_n
        sns.barplot(data=pair_df, x="Count", y=f"{type}_Pair_Formatted", palette="viridis")
        plt.title(f"Top {top_n} Most Frequently Bought {type} Pairs", fontsize=16)
        plt.xlabel("Number of Orders Together", fontsize=13)
        plt.ylabel(f"{type} Pair", fontsize=13)
        for i, v in enumerate(pair_df["Count"].values):
            plt.text(v + 10, i, f'{v}', color='black', va='center', fontsize=11)
        plt.tight_layout()
        plt.show()
        return "PLOTTED"

class CustomerProfileReportTool(Tool):
    name = "customer_profile_report"
    description = (
        "Generates a comprehensive purchase report for a specific customer. "
        "Returns a detailed string summary."
    )
    inputs = {
        "customer_phone": {"type": "string", "description": "The phone number of the customer to analyze."},
    }
    output_type = "string"

    def forward(self, customer_phone: str):
        customer_df = df[df['Customer_Phone'].astype(str) == customer_phone].copy()

        if customer_df.empty:
            return f"No data found for customer phone: {customer_phone}"

        customer_df.sort_values('Delivered_date', inplace=True)
        customer_df['Month'] = customer_df['Delivered_date'].dt.to_period('M')

        customer_name = customer_df['Customer_Name'].iloc[0] if not customer_df.empty else 'N/A'
        brands_bought = customer_df['Brand'].unique().tolist()
        total_brands_bought = len(brands_bought)
        total_unique_skus_bought = customer_df['SKU_Code'].nunique()
        skus_bought = customer_df['SKU_Code'].unique().tolist()

        report_parts = [
            f"## Customer Purchase Profile for {customer_name} ({customer_phone})",
            f"**Customer Branch:** {customer_df['Branch'].iloc[0] if 'Branch' in customer_df.columns and not customer_df.empty else 'N/A'}",
            f"**Total Unique Brands Bought:** {total_brands_bought}",
            f"**Brands Bought:** {', '.join(brands_bought)}",
            f"**Total Order Count:** {customer_df['Order_Id'].nunique() if 'Order_Id' in customer_df.columns else len(customer_df)}",
            f"**Total Unique SKUs Bought:** {total_unique_skus_bought}",
            f"**SKUs Bought:** {', '.join(skus_bought)}",
        ]

        # Brand Level Summary
        report_parts.append("\n### Brand Level Purchase Summary:")
        purchase_summary_by_brand = {}
        for brand in brands_bought:
            brand_df = customer_df[customer_df['Brand'] == brand]
            last_purchase_date = brand_df['Delivered_date'].max().strftime('%Y-%m-%d') if not brand_df.empty else 'N/A'
            total_quantity = brand_df['Delivered Qty'].sum()
            total_spent = brand_df['Total_Amount_Spent'].sum()
            purchase_summary_by_brand[brand] = {
                'Last Purchase Date': last_purchase_date,
                'Total Quantity Bought': total_quantity,
                'Total Amount Spent': round(total_spent, 2)
            }
            report_parts.append(f"- **{brand}**: Last Purchase: {last_purchase_date}, Total Qty: {total_quantity}, Total Spent: {total_spent:,.2f}")

        # Salesman Analysis
        if 'Salesman_Name' in customer_df.columns and 'Order_Id' in customer_df.columns and not customer_df.empty:
            salesman_unique_order_counts = customer_df.groupby('Salesman_Name')['Order_Id'].nunique()
            if not salesman_unique_order_counts.empty and salesman_unique_order_counts.max() > 0:
                most_sold_salesman_name = salesman_unique_order_counts.idxmax()
                most_sold_salesman_count = salesman_unique_order_counts.max()
                salesman_designation = customer_df[customer_df['Salesman_Name'] == most_sold_salesman_name]['Designation'].iloc[0] if 'Designation' in customer_df.columns else 'N/A'
                report_parts.append(f"\n**Top Salesperson:** {most_sold_salesman_name} ({int(most_sold_salesman_count)} orders), Designation: {salesman_designation}")
            else:
                report_parts.append("\n**Top Salesperson:** N/A (No sales data for salesperson)")
        else:
            report_parts.append("\n**Top Salesperson:** N/A (Salesman data missing or incomplete)")

        return "\n".join(report_parts)

class HeuristicNextPurchasePredictionTool(Tool):
    name = "heuristic_next_purchase_prediction"
    description = (
        "Predicts the next likely purchase (SKU level) for a customer based on their historical purchasing intervals. "
        "Returns a string summary of predictions."
    )
    inputs = {
        "customer_phone": {"type": "string", "description": "The phone number of the customer to predict for."},
    }
    output_type = "string"

    def forward(self, customer_phone: str):
        customer_df = df[df['Customer_Phone'].astype(str) == customer_phone].copy()

        if customer_df.empty:
            return f"No data found for customer phone: {customer_phone}. Cannot make heuristic predictions."

        customer_df['Delivered_date'] = pd.to_datetime(customer_df['Delivered_date'], errors='coerce')
        customer_df.sort_values('Delivered_date', inplace=True)
        customer_df['Month'] = customer_df['Delivered_date'].dt.to_period('M')

        last_purchase_date_sku = customer_df.groupby('SKU_Code')['Delivered_date'].max()

        avg_interval_days = {}
        for sku, grp in customer_df.groupby('SKU_Code'):
            dates = grp['Delivered_date'].drop_duplicates().sort_values()
            if len(dates) > 1:
                intervals = dates.diff().dt.days.dropna()
                if not intervals.empty:
                    avg_interval_days[sku] = int(intervals.mean())
                else:
                    avg_interval_days[sku] = np.nan
            else:
                avg_interval_days[sku] = np.nan

        avg_qty_sku = customer_df.groupby(['SKU_Code', 'Month'])['Delivered Qty'].sum().groupby('SKU_Code').mean().round(0)
        avg_spend_sku = customer_df.groupby(['SKU_Code', 'Month'])['Total_Amount_Spent'].sum().groupby('SKU_Code').mean().round(0)
        sku_to_brand = customer_df[['SKU_Code', 'Brand']].drop_duplicates().set_index('SKU_Code')['Brand']

        sku_predictions_df = pd.DataFrame({
            'Last Purchase Date': last_purchase_date_sku.dt.date,
            'Avg Interval Days': pd.Series(avg_interval_days),
            'Expected Quantity': avg_qty_sku,
            'Expected Spend': avg_spend_sku
        }).dropna(subset=['Avg Interval Days'])

        if not sku_predictions_df.empty:
            sku_predictions_df['Next Purchase Date'] = (
                pd.to_datetime(sku_predictions_df['Last Purchase Date']) +
                pd.to_timedelta(sku_predictions_df['Avg Interval Days'], unit='D')
            )
            sku_predictions_df = sku_predictions_df.merge(sku_to_brand.rename('Brand'), left_index=True, right_index=True, how='left')
            sku_predictions_df['Likely Purchase Date'] = sku_predictions_df['Next Purchase Date'].dt.strftime('%Y-%m-%d') + ' (' + sku_predictions_df['Next Purchase Date'].dt.day_name() + ')'
        #else:
        #    return "Not enough historical data to provide detailed SKU purchase predictions for this customer."

        sku_predictions_df = sku_predictions_df.reset_index().rename(columns={
            'index': 'SKU Code',
            'Brand': 'Likely Brand',
        })
        sku_predictions_df = sku_predictions_df.sort_values(
            by='Next Purchase Date', ascending=True
        ).head(3)

        if sku_predictions_df.empty:
            return "No heuristic predictions could be generated for this customer."

        prediction_summary = ["### Heuristic Next Purchase Predictions:"]
        for _, row in sku_predictions_df.iterrows():
            prediction_summary.append(
                f"- **{row['Likely Brand']}** ({row['SKU Code']}): Likely Purchase on {row['Likely Purchase Date']}, "
                f"Expected Quantity: {int(row['Expected Quantity'])}, Expected Spend: {int(row['Expected Spend']):,.0f}"
            )
        return "\n".join(prediction_summary)


class SKURecommenderTool(Tool):
    name = "sku_recommender"
    description = (
        "Generates personalized SKU recommendations for a customer based on a hybrid model. "
        "Returns a string summary of previously purchased and recommended SKUs."
    )
    inputs = {
        "customer_phone": {"type": "string", "description": "The phone number of the customer to recommend for."},
        "top_n": {"type": "integer", "description": "Number of top recommendations to return (default 5).", "required": False, "nullable": True},
    }
    output_type = "string"

    def forward(self, customer_phone: str, top_n: int = 5):
        # Replicate prepare_recommender_data logic
        try:
            user_item_matrix = df.pivot_table(index='Customer_Phone', columns='SKU_Code', values='Redistribution Value', aggfunc='sum', fill_value=0)
            item_similarity = cosine_similarity(user_item_matrix.T)
            item_similarity_df = pd.DataFrame(item_similarity,
                                              index=user_item_matrix.columns,
                                              columns=user_item_matrix.columns)

            item_attributes_cols = ['SKU_Code', 'Brand']
            if 'Branch' in df.columns:
                item_attributes_cols.append('Branch')

            item_attributes = df[item_attributes_cols].drop_duplicates(subset=['SKU_Code']).set_index('SKU_Code')
            for col in ['Brand', 'Branch']:
                if col in item_attributes.columns:
                    item_attributes[col] = item_attributes[col].astype(str).fillna('Unknown')

            encoder = OneHotEncoder(handle_unknown='ignore')
            item_features_encoded = encoder.fit_transform(item_attributes)
            content_similarity = cosine_similarity(item_features_encoded)
            content_similarity_df = pd.DataFrame(content_similarity,
                                                  index=item_attributes.index,
                                                  columns=item_attributes.index)

            common_skus = item_similarity_df.index.intersection(content_similarity_df.index)
            if common_skus.empty:
                return "Recommender system could not be initialized: No common SKUs found between collaborative and content-based models."

            filtered_item_similarity = item_similarity_df.loc[common_skus, common_skus]
            filtered_content_similarity = content_similarity_df.loc[common_skus, common_skus]
            hybrid_similarity = (filtered_item_similarity + filtered_content_similarity) / 2

            sku_brand_map = df[['SKU_Code', 'Brand']].drop_duplicates(subset='SKU_Code').set_index('SKU_Code')

        except Exception as e:
            return f"Error preparing recommender data: {e}"

        # Replicate recommend_skus_brands logic
        if customer_phone not in user_item_matrix.index:
            return f"Customer {customer_phone} not found in the purchase history for recommendations."

        purchased_skus = user_item_matrix.loc[customer_phone]
        purchased_skus = purchased_skus[purchased_skus > 0].index.tolist()

        if not purchased_skus:
            return f"Customer {customer_phone} has no recorded purchases. Cannot generate recommendations."

        valid_purchased_skus = [sku for sku in purchased_skus if sku in hybrid_similarity.columns]
        if not valid_purchased_skus:
            return "No valid purchased SKUs for similarity calculation. Cannot generate recommendations."

        sku_scores = hybrid_similarity[valid_purchased_skus].mean(axis=1)
        sku_scores = sku_scores.drop(index=[s for s in purchased_skus if s in sku_scores.index], errors='ignore')

        if sku_scores.empty:
            return "No new recommendations could be generated for this customer."

        top_skus = sku_scores.sort_values(ascending=False).head(top_n)

        recommendations_df = sku_brand_map.loc[top_skus.index.intersection(sku_brand_map.index)].copy()
        recommendations_df['Similarity_Score'] = top_skus.loc[recommendations_df.index].values
        recommendations_df = recommendations_df.reset_index()

        # Format output
        report_parts = [f"### SKU Recommendations for Customer {customer_phone}:"]

        report_parts.append("\n**Previously Purchased SKUs:**")
        if purchased_skus:
            past_purchases_info = df[df['Customer_Phone'].astype(str) == customer_phone][['SKU_Code', 'Brand']].drop_duplicates()
            for _, row in past_purchases_info.iterrows():
                report_parts.append(f"- {row['SKU_Code']} ({row['Brand']})")
        else:
            report_parts.append("No past purchase data found for this customer.")

        report_parts.append("\n**Recommended SKUs:**")
        if not recommendations_df.empty:
            for _, row in recommendations_df.iterrows():
                report_parts.append(f"- {row['SKU_Code']} ({row['Brand']}) - Similarity: {row['Similarity_Score']:.4f}")
            report_parts.append("\n*A higher 'Similarity Score' indicates a stronger recommendation.*")
        else:
            report_parts.append("No new recommendations could be generated for this customer.")

        return "\n".join(report_parts)


# Instantiate ALL tools
head_tool = HeadTool()
tail_tool = TailTool()
info_tool = InfoTool()
describe_tool = DescribeTool()
histogram_tool = HistogramTool()
scatter_tool = ScatterPlotTool()
correlation_tool = CorrelationTool()
pivot_tool = PivotTableTool()
filter_rows_tool = FilterRowsTool()
groupby_tool = GroupByAggregateTool()
sort_tool = SortTool()
topn_tool = TopNTool()
crosstab_tool = CrosstabTool()
linreg_tool = LinRegEvalTool()
predict_tool = PredictLinearTool()
rf_tool = RFClassifyTool()
final_answer_tool = FinalAnswerTool()
insights_tool = InsightsTool()

# New tools
plot_bar_chart_tool = PlotBarChartTool()
plot_line_chart_tool = PlotLineChartTool()
plot_dual_axis_line_chart_tool = PlotDualAxisLineChartTool()
brand_sku_pair_analysis_tool = BrandSKUPairAnalysisTool()
customer_profile_report_tool = CustomerProfileReportTool()
heuristic_next_purchase_prediction_tool = HeuristicNextPurchasePredictionTool()
sku_recommender_tool = SKURecommenderTool()


# Construct the tool list
tools = [
    head_tool, tail_tool, info_tool, describe_tool, histogram_tool, scatter_tool,
    correlation_tool, pivot_tool, filter_rows_tool, groupby_tool, sort_tool,
    topn_tool, crosstab_tool, linreg_tool, predict_tool, rf_tool,
    final_answer_tool, insights_tool,
    # Add new tools
    plot_bar_chart_tool, plot_line_chart_tool, plot_dual_axis_line_chart_tool,
    brand_sku_pair_analysis_tool, customer_profile_report_tool,
    heuristic_next_purchase_prediction_tool, sku_recommender_tool,
]

# Initialize LiteLLMModel
model = LiteLLMModel(
    model_id="openrouter/meta-llama/llama-4-maverick",
    temperature=0.2,
    api_key= "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", # Use os.environ.get for robustness
    #api_key=os.environ.get("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"), # Use os.environ.get for robustness
    additional_kwargs={
        "custom_llm_provider": "openrouter",
        "max_tokens": 1024,
        "max_completion_tokens": 1024,
    },
)

In [ ]:
# Create the CodeAgent with the complete toolset
agent = CodeAgent(
    tools=tools,
    model=model,
    description="""
You are a Grandmaster Data Science assistant. Two pandas DataFrames are loaded:
- `df`: The main sales data, containing columns like 'Brand', 'SKU_Code', 'Customer_Name', 'Customer_Phone', 'Delivered_date', 'Redistribution Value', 'Delivered Qty', 'Order_Id', 'Month', 'Total_Amount_Spent'.
- `PRED_DF`: Contains model-based purchase predictions, with columns like 'Customer_Phone', 'Next Brand Purchase', 'Next Purchase Date', 'Expected Spend', 'Expected Quantity', 'Probability', 'Suggestion'.

You have access to these tools:
1) head(n) – Show first n rows of `df`.
2) tail(n) – Show last n rows of `df`.
3) info() – Return `df.info()` as string.
4) describe(column) – Summary stats for a column or all of `df`.
5) histogram(column, bins) – Plot histogram of a numeric column in `df`.
6) scatter_plot(column_x, column_y) – Plot scatter of two numeric columns in `df`.
7) correlation(method='pearson') – Compute correlation matrix of numeric columns in `df`.
8) pivot_table(index, columns, values, aggfunc) – Create pivot table from `df`.
9) filter_rows(column, operator, value) – Filter `df` rows. Returns the filtered DataFrame.
10) groupby_agg(group_columns, metric_column, aggfunc) – Group `df` and aggregate. Returns the aggregated DataFrame.
11) sort(column, ascending) – Sort `df` by column. Returns the sorted DataFrame.
12) top_n(metric_column, n, group_columns=None, ascending=False) – Top/Bottom n by metric (optional grouping, specify `ascending=True` for bottom N) from `df`. Returns the result DataFrame.
13) crosstab(row, column, aggfunc=None, values=None) – Crosstab between categories in `df`.
14) linreg_eval(feature_columns, target_column, test_size=0.2) – Train/test + LinearRegression on `df`, return R².
15) predict_linear(feature_columns, target_column, new_data) – Fit LinearRegression on `df`, predict new row.
16) rf_classify(feature_columns, target_column, test_size=0.2, n_estimators=100) – RF classification on `df`, return report.
17) final_answer(text) – Return a direct final answer to the user as string.
18) insights() – Compute overall sales-dataset insights and actionable recommendations. No arguments.

**New Visualization Tools:**
19) plot_bar_chart(data, x_column, y_column, title, xlabel=None, ylabel=None, horizontal=False, sort_by_x_desc=True) – Plots a bar chart from a DataFrame.
20) plot_line_chart(data, x_column, y_column, title, hue_column=None, xlabel=None, ylabel=None) – Plots a line chart for time-series data.
21) plot_dual_axis_line_chart(data, x_column, y1_column, y2_column, title, xlabel=None, y1_label=None, y2_label=None) – Plots two line charts on a dual y-axis.

**New Analysis & Reporting Tools:**
22) brand_sku_pair_analysis(type, top_n=10) – Analyzes and plots most frequently co-purchased 'Brand' or 'SKU_Code' pairs.
23) customer_profile_report(customer_phone) – Generates a comprehensive purchase report for a specific customer from `df`.
24) heuristic_next_purchase_prediction(customer_phone) – Predicts next likely purchase (SKU level) for a customer from `df` based on historical intervals.
25) sku_recommender(customer_phone, top_n=5) – Generates personalized SKU recommendations for a customer using a hybrid model (from `df` and `PRED_DF`).

**Instructions for tool usage:**
- When the user asks for “summary,” “data insights,” “actionable recommendations,” or a general overview of performance, prioritize calling `insights()`.
- For specific data queries requiring visualization, use `groupby_agg` or `top_n` first to prepare the data, then use `plot_bar_chart`, `plot_line_chart`, or `plot_dual_axis_line_chart` to visualize the result.
- For detailed customer information, use `customer_profile_report`.
- For specific next-purchase predictions, use `heuristic_next_purchase_prediction` or refer to `PRED_DF` directly if model-based predictions are requested.
- For product recommendations, use `sku_recommender`.
- Always aim to provide actionable insights where possible.
- Otherwise, pick exactly one tool that best fits and return one line of Python calling it (using named arguments). No explanations, no extra output—just the function call.
""",
    additional_authorized_imports=["pandas", "datetime", "io", "matplotlib.pyplot", "seaborn", "numpy", "itertools", "collections", "sklearn.metrics.pairwise", "sklearn.preprocessing"]
)

print("🧠 Your Grandmaster Data‐Science Agent is ready!")
print("Ask anything about `df` or `PRED_DF`. (Type 'exit' or 'quit' to stop.)\n")

# Main interactive loop
while True:
    user_prompt = input("➡️ Your request: ").strip()
    if user_prompt.lower() in ("exit", "quit"):
        print("👋 Goodbye!")
        break

    full_prompt = f"""
You are a Grandmaster Data Science assistant helping a human analyze a pandas DataFrame named `df` and `PRED_DF`.
You have access to the following tools:
{agent.description} # This will include the tool descriptions from the agent itself

Each tool has a defined purpose and must be called using named arguments only.
🧠 Your task:
Based on the user request, decide which tool best fits.
Then return ONLY one valid Python line calling that tool:
• ✅ Example → top_n(metric_column="revenue", n=10, group_columns="region")
• ❌ No comments, no explanations, no extra output
The DataFrames `df` and `PRED_DF` are already loaded and ready.
User request: {user_prompt!r}
Tool call:
"""
    try:
        tool_call = agent.run(full_prompt).strip()
        print("\n🔧 My Response:\n")
        print(tool_call)

        # Parse tool name from generated line
        tool_name = tool_call.split("(")[0].strip()

        # Manually create tool_dispatch for direct execution in this script
        # In a real smolagents environment, this dispatch is handled internally
        tool_dispatch = {tool.name: tool.forward for tool in tools}

        result = eval(tool_call, globals(), tool_dispatch)

        if isinstance(result, pd.DataFrame):
            display(result)
        elif isinstance(result, (pd.Series, str)):
            print(result)
        else:
            print("✅ Result:", result)
        print("\n" + ("─" * 60) + "\n")
    except Exception as e:
        #print(f"{str(e)}")
        #print(f"❌ Error during tool execution: {str(e)}")
        #print("Finished")
        print("\n" + ("─" * 60) + "\n")


🧠 Your Grandmaster Data‐Science Agent is ready!
Ask anything about `df` or `PRED_DF`. (Type 'exit' or 'quit' to stop.)



╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a Grandmaster Data Science assistant helping a human analyze a pandas DataFrame named `df` and          │
│ `PRED_DF`.                                                                                                      │
│ You have access to the following tools:                                                                         │
│                                                                                                                 │
│ You are a Grandmaster Data Science assistant. Two pandas DataFrames are loaded:                                 │
│ - `df`: The main sales data, containing columns like 'Brand', 'SKU_Code', 'Customer_Name', 'Customer_Phone',    │
│ 'Delivered_date', 'Redistribution Value', 'Delivered Qty', 'Order_Id', 'Month', 'Total_Amount_Spent'.           │
│ - `PRED_DF`: Contains model-based purchase predictions, with columns like 'Customer_Phone', 'Next Brand         │
│ Purchase', 'Next Purchase Date', 'Expected Spend', 'Expected Quantity', 'Probability', 'Suggestion'.            │
│                                                                                                                 │
│ You have access to these tools:                                                                                 │
│ 1) head(n) – Show first n rows of `df`.                                                                         │
│ 2) tail(n) – Show last n rows of `df`.                                                                          │
│ 3) info() – Return `df.info()` as string.                                                                       │
│ 4) describe(column) – Summary stats for a column or all of `df`.                                                │
│ 5) histogram(column, bins) – Plot histogram of a numeric column in `df`.                                        │
│ 6) scatter_plot(column_x, column_y) – Plot scatter of two numeric columns in `df`.                              │
│ 7) correlation(method='pearson') – Compute correlation matrix of numeric columns in `df`.                       │
│ 8) pivot_table(index, columns, values, aggfunc) – Create pivot table from `df`.                                 │
│ 9) filter_rows(column, operator, value) – Filter `df` rows. Returns the filtered DataFrame.                     │
│ 10) groupby_agg(group_columns, metric_column, aggfunc) – Group `df` and aggregate. Returns the aggregated       │
│ DataFrame.                                                                                                      │
│ 11) sort(column, ascending) – Sort `df` by column. Returns the sorted DataFrame.                                │
│ 12) top_n(metric_column, n, group_columns=None, ascending=False) – Top/Bottom n by metric (optional grouping,   │
│ specify `ascending=True` for bottom N) from `df`. Returns the result DataFrame.                                 │
│ 13) crosstab(row, column, aggfunc=None, values=None) – Crosstab between categories in `df`.                     │
│ 14) linreg_eval(feature_columns, target_column, test_size=0.2) – Train/test + LinearRegression on `df`, return  │
│ R².                                                                                                             │
│ 15) predict_linear(feature_columns, target_column, new_data) – Fit LinearRegression on `df`, predict new row.   │
│ 16) rf_classify(feature_columns, target_column, test_size=0.2, n_estimators=100) – RF classification on `df`,   │
│ return report.                                                                                                  │
│ 17) final_answer(text) – Return a direct final answer to the user as string.                                    │
│ 18) insights() – Compute overall sales-dataset insight

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  sku_recommender(customer_phone='8060733751')                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: Customer 8060733751 not found in the purchase history for recommendations.

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("To keep customer 8039230738 (Mummy Ada) buying more, consider the following strategies based on    
  the customer's profile and purchase history:\n1. **Promote INDOMIE products**: Since INDOMIE is the brand she    
  purchases most frequently and has spent the most on (934,600.00), offering discounts or loyalty rewards on       
  INDOMIE products could encourage more purchases.\n2. **Stock Availability**: Ensure that the SKUs she            
  frequently buys (e.g., 10000002, 10000391, 10000400) are always in stock. Notify her about restocks or           
  availability through her preferred channel.\n3. **Personalized Offers**: Since her last purchase was on          
  2025-03-01, and the predicted next purchase is on 2026-03-19, plan targeted promotions around this time.         
  Consider offering bundle deals on INDOMIE and related products.\n4. **Engage with her preferred salesperson**:   
  The top salesperson for her has been EMMANUEL JACKSON. Maintaining a good relationship through him could help    
  in understanding her needs better and tailoring offerings.\n5. **Loyalty Program**: Enroll her in a loyalty      
  program that rewards her for repeat purchases, especially for brands like INDOMIE.\n6. **Feedback Collection**:  
  Collect feedback through EMMANUEL JACKSON or other means to understand her preferences and improve product       
  offerings.")                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: To keep customer 8039230738 (Mummy Ada) buying more, consider the following strategies based on
the customer's profile and purchase history:
1. **Promote INDOMIE products**: Since INDOMIE is the brand she purchases most frequently and has spent the most on
(934,600.00), offering discounts or loyalty rewards on INDOMIE products could encourage more purchases.
2. **Stock Availability**: Ensure that the SKUs she frequently buys (e.g., 10000002, 10000391, 10000400) are always
in stock. Notify her about restocks or availability through her preferred channel.
3. **Personalized Offers**: Since her last purchase was on 2025-03-01, and the predicted next purchase is on 
2026-03-19, plan targeted promotions around this time. Consider offering bundle deals on INDOMIE and related 
products.
4. **Engage with her preferred salesperson**: The top salesperson for her has been EMMANUEL JACKSON. Maintaining a 
good relationship through him could help in understanding her needs better and tailoring offerings.
5. **Loyalty Program**: Enroll her in a loyalty program that rewards her for repeat purchases, especially for 
brands like INDOMIE.
6. **Feedback Collection**: Collect feedback through EMMANUEL JACKSON or other means to understand her preferences 
and improve product offerings.

[Step 5: Duration 3.02 seconds| Input tokens: 30,429 | Output tokens: 970]


🔧 Agent chose call:

To keep customer 8039230738 (Mummy Ada) buying more, consider the following strategies based on the customer's profile and purchase history:
1. **Promote INDOMIE products**: Since INDOMIE is the brand she purchases most frequently and has spent the most on (934,600.00), offering discounts or loyalty rewards on INDOMIE products could encourage more purchases.
2. **Stock Availability**: Ensure that the SKUs she frequently buys (e.g., 10000002, 10000391, 10000400) are always in stock. Notify her about restocks or availability through her preferred channel.
3. **Personalized Offers**: Since her last purchase was on 2025-03-01, and the predicted next purchase is on 2026-03-19, plan targeted promotions around this time. Consider offering bundle deals on INDOMIE and related products.
4. **Engage with her preferred salesperson**: The top salesperson for her has been EMMANUEL JACKSON. Maintaining a good relationship through him could help in understanding her needs better 